In [2]:
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely as shp
import rasterio as rio 
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
# Directories
data_dir = os.path.join(os.getcwd(),"data") # path to data-folder with aux data
proc_tiles_path = os.path.join(os.path.join(data_dir,"VN_processing_polygons")) # path to processing tiles
country_bounds_path = os.path.join(data_dir,"VN_country_bounds") # path to country bounds
osm_sl_path = os.path.join(os.path.join(data_dir,"VN_osm_coastline")) # path to reference shoreline
buffer_path = os.path.join(os.path.join(data_dir,"VN_buffer_5km"))

# Params
crs = "EPSG:3857"                   

In [7]:
# read/ create aux data 
proc_tiles = gpd.read_file(proc_tiles_path).to_crs(crs)

In [151]:
# Number of images used for processing coastlines of entire Vietnam from 1984-May 2
aq_sums = []
aq_annual = dict.fromkeys(range(1987,2021))
aq_annual(aq_annual[1987].append(5))
print(aq_annual)
for i in proc_tiles.id:
    tile_name = "P"+str(i).zfill(2) # name of processing tile
    folder_path = os.path.join(data_dir,tile_name) # path to mndwi rasters and meta
    meta_path = os.path.join(data_dir,tile_name,tile_name+".csv")
    meta = pd.read_csv(meta_path)
    meta = meta.drop(columns=['system:index','.geo'])
    meta = meta.transpose().rename(columns={0:tile_name})
    meta['year'] = meta.index
    meta = meta.reset_index(drop=True)
    aq_sum = np.sum(meta[tile_name])
    aq_sums.append(aq_sum)
    for year in meta.year:
        print(year)
        y = str(year)
        aq = meta[meta.year==y][tile_name][0]
        print(aq)
        break
    for year in aq_annual:
        aq_annual[year]
        break
    break
    #for year in meta.year:
    #    print(meta[meta.year==year])
    #break
all_images = np.sum(aq_sums) # 26,109 images have been used for the entire analysis

AttributeError: 'NoneType' object has no attribute 'append'